# Milvus Similarity Search for encoded logs

Example Usage: | search ... | table text
| fit MLTKContainer algo=llm_rag_milvus_search collection_name=notable_events_384_transformers embedder_name="all-MiniLM-L6-v2" use_local=1 top_k=5 text into app:llm_rag_milvus_search
| table text predicted_Results predicted_Fields
| eval predicted_Results=split(predicted_Results,"|"), predicted_Fields=split(predicted_Fields,"|")
| mvexpand predicted_Results
| mvexpand predicted_Fields
| dedup predicted_Results

Note: By default every time you save this notebook the stages main cells are exported into a python module which can then get invoked by Splunk's MLTK SPL commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the DSDL app for more information about this workflow.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import time
import pymilvus
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.26.4
pandas version: 2.2.0


In [3]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [4]:
df, param = stage('llm_rag_milvus_search')

## Stage 2 - create and initialize a model

In [6]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
        
    pk_type=DataType.VARCHAR        
    embedding_type=DataType.FLOAT_VECTOR
    
    try:
        collection_name=param['options']['params']['collection_name'].strip('\"')
    except:
        collection_name="default_collection"
    
    print("start connecting to Milvus")
    # this hostname may need changing to a specific local docker network ip address depending on docker configuration
    connections.connect("default", host="milvus-standalone", port="19530")

    collection_exists = utility.has_collection(collection_name)
    
    if collection_exists:
        print(f"The collection {collection_name} already exists")
        collection = Collection(collection_name)
        collection.load()
    else:
        print(f"The collection {collection_name} does not exist")
        raise Exception("The collection {collection_name} does not exist. Create it by sending data to a collection with that name using the push_to_milvus algo.")
    
    model['collection']=collection
    model['collection_name']=collection_name
    return model

In [7]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

start connecting to Milvus
The collection internal_log_256 already exists
{'collection': <Collection>:
-------------
<name>: internal_log_256
<description>: dsdl schema for internal_log_256
<schema>: {'auto_id': True, 'description': 'dsdl schema for internal_log_256', 'fields': [{'name': '_key', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'embeddings', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 256}}, {'name': 'label', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000}}], 'enable_dynamic_field': False}
, 'collection_name': 'internal_log_256'}


## Stage 3 - fit the model

In [12]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [13]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

{'message': 'model trained'}


## Stage 4 - apply the model

In [14]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    use_local= int(param['options']['params']['use_local'])
    try:
        embedder_name=param['options']['params']['embedder_name'].strip('\"')
    except:
        embedder_name = 'all-MiniLM-L6-v2'
    if use_local:
        embedder_name = f'/srv/app/model/data/{embedder_name}'
        print("Using local embedding model checkpoints") 
    transformer_embedder = HuggingFaceEmbedding(model_name=embedder_name)
    
    try:
       top_k=int(param['options']['params']['top_k'])
    except:
        top_k=3
        
    try:
        splitter=param['options']['params']['splitter']
    except:
        splitter="|"
    
    text_column = df['text'].astype(str).tolist()

    vector_column = []
    for text in text_column:
        vector_column.append(transformer_embedder.get_text_embedding(text))
    
    search_params = {
        "metric_type": "L2",
        "params": {"nprobe": 10},
    }
    output_fields = [item.name for item in model['collection'].schema.fields]
    output_fields.remove('embeddings')
    results = model['collection'].search(data=vector_column, anns_field="embeddings", param=search_params, limit=top_k, output_fields=output_fields)
    l = []
    f = []
    output_fields.remove('label')
    for result in results:
        x = ''
        y = ''
        for r in result:
            t = {}
            # t['data'] = r.entity.get('label')
            for field in output_fields:
                t[field] = r.entity.get(field)
            x += str(t)
            x += splitter
            y += r.entity.get('label')
            y += splitter
        xs = x.rstrip(splitter)
        ys = y.rstrip(splitter)
        l.append(ys) 
        f.append(xs)
    
    
    cols = {"Results": l, "Fields": f}
    returns = pd.DataFrame(data=cols)
    return returns

In [15]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

     index
0        0
1        1
2        2
3        3
4        4
..     ...
995    995
996    996
997    997
998    998
999    999

[1000 rows x 1 columns]


## Stage 5 - save the model

In [16]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [17]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [18]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DSDL app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [18]:
from dsdlsupport import SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [19]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [20]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


In [21]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [22]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


## End of Stages
All subsequent cells are not tagged and can be used for further freeform code